In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = "../data/labelled_data/"
test = pd.read_csv(path + "test_subset.csv")
train_1 = pd.read_csv(path + "train_first_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_2 = pd.read_csv(path + "train_second_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_3 = pd.read_csv(path + "train_third_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_1["label"] = pd.to_numeric(train_1["label"], errors='coerce')
train = pd.concat([train_1,train_2,train_3],axis=0)
val = pd.read_csv(path + "val_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]

test = test.reset_index()
train=train.reset_index()
val = val.reset_index()

train.head()

In [ ]:
for data in [train_1,train_2,train_3,train,val,test]:
    print("Total size = {}".format(data.shape[0]))
    print("Noise = {} ".format(data.loc[(data.label == 0.0) & (data.new_aspect_1.isnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with no adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.notnull())].shape[0]))
    print("")

##### Preprocessing

2 data sets for modelling

1. Use this original data without noise
2. Create new dataset --> Filter the phrase column (punctuations/hashtags) then combine the aspects into 1 & aggregate the sentiment scores (label)

Question : when aggregating use majority? but what if i had alot of noise eg. 0 0 0 0 0 and only 1 -1 then using majority voting will only get 0 not -1? And also mean doesnt work either

Dataset 1

In [ ]:
# train
noise = train.loc[(train.label == 0.0) & (train.new_aspect_1.isnull()) & (train.new_aspect_2.isnull())].index
train1 = train.loc[~train.index.isin(noise)]
print(train1.shape)

# test
noise = val.loc[(val.label == 0.0) & (val.new_aspect_1.isnull()) & (val.new_aspect_2.isnull())].index
val1 = val.loc[~val.index.isin(noise)]
print(val1.shape)

#val 
noise = test.loc[(test.label == 0.0) & (test.new_aspect_1.isnull()) & (test.new_aspect_2.isnull())].index
test1 = test.loc[~test.index.isin(noise)]
print(test1.shape)

In [ ]:
print(train1.label.value_counts())
print(test1.label.value_counts())
print(val1.label.value_counts())

Dataset 2 **USE THIS**

In [ ]:
def sgn(x):
    if x < 0:
        return -1
    elif x == 0:
        return 0
    else:
        return 1

In [ ]:
# filter out phrases with hashtags & those with character length <=5 then get the majority (might not make sense)
# train
train2_filtered = train.loc[~train.index.isin(train.loc[(train.phrase.str.contains("#")) | (train.phrase.str.len() <= 5)].index)]
print("Number of rows after filtering (Train) = {}".format(train2_filtered.shape[0]))
train2 = train2_filtered.groupby(["review_title","account_name","aspect"]).agg({"phrase": " ".join,
                                                                                "label": "mean"})
train2 = train2.reset_index()
train2.label = train2.label.map(sgn)


# test
test2_filtered = test.loc[~test.index.isin(test.loc[(test.phrase.str.contains("#")) | (test.phrase.str.len() <= 5)].index)]
print("Number of rows after filtering (Test) = {}".format(test2_filtered.shape[0]))
test2 = test2_filtered.groupby(["review_title","account_name","aspect"]).agg({"phrase": " ".join,
                                                                              "label": "mean"})
test2 = test2.reset_index()
test2.label = test2.label.map(sgn)

#val 
val2_filtered = val.loc[~val.index.isin(val.loc[(val.phrase.str.contains("#")) | (val.phrase.str.len() <= 5)].index)]
print("Number of rows after filtering (Val) = {}".format(val2_filtered.shape[0]))
val2 = val2_filtered.groupby(["review_title","account_name","aspect"]).agg({"phrase": " ".join,
                                                                              "label": "mean"})
val2 = val2.reset_index()
val2.label = val2.label.map(sgn)

In [ ]:
train2.head()

In [ ]:
# use train & val as training
train_val2 = pd.concat([train2,val2],axis=0)
train_val2.shape

In [ ]:
print(train_val2.label.value_counts())
print(test2.label.value_counts())

###### Text Vectorisation --> used on data2
1. BoW
2. Tf-idf
3. Fasttext - can handle oov words https://www.quora.com/How-does-fastText-output-a-vector-for-a-word-that-is-not-in-the-pre-trained-model


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
import re
from tqdm import tqdm

In [ ]:
# Text cleaning function for gensim fastText word embeddings in python
# remove numbers
def process_text(document):     
        # Remove extra white space from text
        document = re.sub(r'\s+', ' ', document, flags=re.I)     
        # Remove all the special characters from text
        document = re.sub(r'\W', ' ', str(document))
        # Remove all single characters from text
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        # Remove numbers
        document = re.sub("(\s\d+)","",document) 
        # Converting to Lowercase
        document = document.lower()
        # Word tokenization       
        tokens = document.split()
        # Lemmatization using NLTK
        lemma_txt = [stemmer.lemmatize(word) for word in tokens]
        # Remove stop words
        lemma_no_stop_txt = [word for word in lemma_txt if word not in stopwords]
        # Drop words 
        tokens = [word for word in tokens if len(word) > 3]            
        clean_txt = ' '.join(lemma_no_stop_txt)
        return clean_txt

In [ ]:
bow = CountVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3),preprocessor=process_text)
bow_train = bow.fit_transform(train_val2.phrase)
bow_test = bow.transform(test2.phrase)

In [ ]:
tfidf = TfidfVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3),preprocessor=process_text)
tfidf_train = tfidf.fit_transform(train_val2.phrase)
tfidf_test = tfidf.transform(test2.phrase)

In [ ]:
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

# Preprocess data for fasttext word embeddings training
train_clean = [process_text(sentence) for sentence in tqdm(train_val2.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
train_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(train_clean)]

# Preprocess data for fasttext word embeddings testing data
test_clean = [process_text(sentence) for sentence in tqdm(test2.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
test_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(test_clean)]

# Train Fasttext word embeddings on training data
# Defining values for parameters
embedding_size = 300
window_size = 5
min_word = 5
down_sampling = 1e-2
 

fast_Text_model = FastText(train_word_tokens,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers = 4,
                      sg=1,
                      iter=100)

In [ ]:
# Save fastText gensim model
fast_Text_model.save("ftmodel/ft_model_train")
# Load saved gensim fastText model
fast_Text_model = Word2Vec.load("ftmodel/ft_model_train")

In [ ]:
# convert sentence --> average(each word embedding)
fasttext_train = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in train_word_tokens]
# convert sentence --> average(each word embedding)
fasttext_test = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in test_word_tokens]


In [ ]:
fasttext_train_processed = []
for row in fasttext_train:
    try:
        fasttext_train_processed.append(list(row))
    except:
        fasttext_train_processed.append([0]*300)

fasttext_test_processed = []
for row in fasttext_test:
    try:
        fasttext_test_processed.append(list(row))
    except:
        fasttext_test_processed.append([0]*300)


In [ ]:
fasttext_train_processed = np.array(fasttext_train_processed).reshape(-1,300)
fasttext_test_processed = np.array(fasttext_test_processed).reshape(-1,300)

##### Modelling

1. Naive Bayes/Support Vector Machines (NBSVM)/Logistic Regression
2. Fasttext
3. BERT 

(Baseline) 1. Naive Bayes -Support Vector Machines (NBSVM)- Logistic Regression

In [ ]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

logreg = LogisticRegression()
nb = MultinomialNB()
svm = SVC()

all_train = [bow_train,tfidf_train,fasttext_train_processed]
all_test = [bow_test,tfidf_test,fasttext_test_processed]
all_vectorisers = ["BoW","TfIdf","FastText"]

for train,test,vectoriser in zip(all_train,all_test,all_vectorisers):
    logreg.fit(train,train_val2.label)
    
    print("Logistic Regression + " + vectoriser)
    print("Training Data")
    print(classification_report(train_val2.label, logreg.predict(train)))
    print("Test Data")
    print(classification_report(test2.label, logreg.predict(test))) 
    
    svm.fit(train,train_val2.label)
    
    print("SVM + " + vectoriser)
    print("Training Data")
    print(classification_report(train_val2.label, svm.predict(train)))
    print("Test Data")
    print(classification_report(test2.label, svm.predict(test))) 

    # Fasttest dont work w NaiveBayes
    nb.fit(train,train_val2.label)
    print("Naive Bayes + "+ vectoriser)
    print("Training Data")
    print(classification_report(train_val2.label, nb.predict(train)))
    print("Test Data")
    print(classification_report(test2.label, nb.predict(test)))    